# Lab Sheet 8: Deep Learning handwritten digits classification with CNNs

This lab is about creating a (moderately) **deep convolutional neural network (CNN)** to classify digits from the **MNIST dataset**.

In this notebook, we'll use **Tensorflow** library with the  **Keras** frontend.  

Since this is new, this notebook **can be mostly run as is in 1) to 4)**. Once you have run it, please try the following:
## Task a)
**Increase** the number of **convolutional layers**. See details in section 2).
## Task b)
Test the notebook **with and without GPU or TPU** (in the Menu under *Runtime* ➡︎ *Change Runtime Type*).


## Task c)
Optional: **Vertex AI**, ([here](https://cloud.google.com/ai-platform) is some general information)




# MNIST digit classification with a CNN

First, we need a number of **imports**.

In [ ]:
!pip install tensorflow

In [ ]:
%matplotlib inline

import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, MaxPooling2D, Conv2D
#from tensorflow.keras.estimator import model_to_estimator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K

from distutils.version import LooseVersion as LV
from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# 1) Loading and preparing data

Now we load the MNIST or [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) dataset. We will create a one-hot encoded version of the labels, which works better for training.

In [ ]:
from tensorflow.keras.datasets import mnist, fashion_mnist
# (X_train, y_train), (X_test, y_test) = mnist.load_data()
# The following line loads the Fashion-MNIST instead of the MNIST dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
nb_classes = 10 # digits 0-9 or different clothing types

X_train = X_train.astype('float32') # this is the normal type for Neural Networks
X_test = X_test.astype('float32') # a good compromise between efficiency and resolution
X_train /= 255 # reduce the range to [0,1]
X_test /= 255 # also for the test set

# one-hot encoding:
Y_train = to_categorical(y_train, nb_classes) # one-hot means that each class corresponds to a vector dimension
Y_test = to_categorical(y_test, nb_classes) # that is set to 1, all others are set to 0

print() # print some info on the data
print('MNIST data loaded: train:', len(X_train), 'test:', len(X_test))
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('Y_train:', Y_train.shape)

We have to do a bit of tensor manipulation, so that the data matches the shape expected by the 2D convolutional layer implementation in tensorflow: # of images, height, width, channels.

In [ ]:
# input image dimensions
img_rows, img_cols = 28, 28

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

print('X_train:', X_train.shape)
print('X_test:', X_test.shape)

# 2) Creating the CNN model

Now we are ready to create a convolutional model.

 * The `Convolution2D` layers operate on 2D matrices so we input the digit images directly to the model.  
 * The `MaxPooling2D` layer reduces the spatial dimensions, that is, makes the image smaller.
 * The `Flatten` layer flattens the 2D matrices into vectors, so we can then switch to  `Dense` layers as in standard neural network.

See https://keras.io/layers/convolutional/, https://keras.io/layers/pooling/ for more information.



In [ ]:
# number of convolutional filters to use
nb_filters = 32
# convolution kernel size
kernel_size = (3, 3)
# size of pooling area for max pooling
pool_size = (2, 2)

model = Sequential()

# Convolutional layer(s)
model.add(Conv2D(nb_filters, kernel_size,
                 padding='valid',
                 input_shape=input_shape,
                 activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(rate=0.25))

# Dense layers
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(units=nb_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())




In [ ]:
!pip install pydot
!apt-get install graphviz -y

In [ ]:
SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

## Task a)
Once you have tested the whole notebook, increase the number of convolutional layers. Try 2 and 3 layers.
This can be achieved by adding another `Conv2D` layer after the first one with `model.add()`.
This can can have the same arguments as the first one. The shapes of the layers are determined automatically, as you can see in the model summary.
What is the effect on the training time and accuracy?

# 3) Training

Now we **train** the CNN **model**.

This is a relatively complex model, so training takes a short while, but it **benefits** considerably from a **GPU** or **TPU**.

In [ ]:
%%time

epochs = 15 # with CPU one epoch takes about 15 seconds (on MNIST)
# with GPU one epoch takes only a few seconds, depending on the number of layers

history = model.fit(X_train,
                    Y_train,
                    epochs=epochs,
                    batch_size=128,
                    verbose=2)

We can now **plot** the training loss and accuracy.

In [ ]:
plt.figure(figsize=(5,3))
plt.plot(history.epoch, history.history['loss'])
plt.title('loss')

plt.figure(figsize=(5,3))
plt.plot(history.epoch, history.history['accuracy'])
plt.title('accuracy');

If we are happy with the model, let's save it.

In [ ]:
model.save('model.keras')

# 4) Inference

We now **use the trained model** on the test set. The test accuracy will be almost 99% for MNIST and 91% for Fasion-MNIST.  

You can compare your MNIST result with the state-of-the art [here](http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html#4d4e495354).  Even more results can be found [here](http://yann.lecun.com/exdb/mnist/).

In [ ]:
%%time
scores = model.evaluate(X_test, Y_test, verbose=2)
print("{}: {:.2f}%".format(model.metrics_names[1], scores[1]*100))

We can again take a closer look on the results. We begin by defining
a helper function to **show the failure cases** of our classifier.

In [ ]:
def show_failures(predictions, trueclass=None, predictedclass=None, maxtoshow=10):
    rounded = np.argmax(predictions, axis=1)
    errors = rounded!=y_test
    print('Showing max', maxtoshow, 'first failures. '
          'The predicted class is shown first and the correct class in parenthesis.')
    ii = 0
    plt.figure(figsize=(maxtoshow, 1))
    for i in range(X_test.shape[0]):
        if ii>=maxtoshow:
            break
        if errors[i]:
            if trueclass is not None and y_test[i] != trueclass:
                continue
            if predictedclass is not None and rounded[i] != predictedclass:
                continue
            plt.subplot(1, maxtoshow, ii+1)
            plt.axis('off')
            if K.image_data_format() == 'channels_first':
                plt.imshow(X_test[i,0,:,:], cmap="gray")
            else:
                plt.imshow(X_test[i,:,:,0], cmap="gray")
            plt.title("%d (%d)" % (rounded[i], y_test[i]))
            ii = ii + 1

Here are the first 10 test examples the CNN **classified to a wrong class**:

In [ ]:
predictions = model.predict(X_test)

show_failures(predictions)

Note that the labels for Fashion-MNIST correspond to 0: T-shirt/top, 1: Trouser, 2: Pullover, 3: Dress, 4: Coat, 5: Sandal, 6: Shirt, 7: Sneaker, 8: Bag, 9: Ankle boot.

We can use `show_failures()` to inspect failures in more detail. For example, I get some cases in which the **true class was "6" but the prediction a "0"**.

In [ ]:
show_failures(predictions, trueclass=6, predictedclass=0)

# 5) Vertex AI (Optional)

If you are interested in Vertex AI, have a look here: https://cloud.google.com/vertex-ai